In [24]:
import os
import json
import re
import scipy.io as spio
import numpy as np
from json_tricks.np import dump, dumps, load, loads
from mat2py import loadmat
import cPickle as pkl
import scipy.io

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]
# 
# class StimInfo:
#     def _init_(self, stimid=None, trials=None, frames=None, frames_sec=None, stim_on_idx=None):
#         self.stimid = stimid #''
#         self.trials = trials # []
#         self.frames = frames # []
#         self.frames_sec = frames_sec # []
#         self.stim_on_idx = stim_on_idx #[]
# 

class StimInfo:
    def _init_(self):
        self.stimid = ''
        self.trials = []
        self.frames = []
        self.frames_sec = []
        self.stim_on_idx = []

def serialize_json(instance=None, path=None):
    dt = {}
    dt.update(vars(instance))

In [25]:
source = '/nas/volume1/2photon/projects'
# experiment = 'scenes'
# session = '20171003_JW016'
# acquisition = 'FOV1'
# functional_dir = 'functional'

experiment = 'gratings_phaseMod'
session = '20171009_CE059'
acquisition = 'FOV1_zoom3x'
functional_dir = 'functional'
 
# ================================================================================
# frame info:
# ================================================================================
#first_frame_on = 50
stim_on_sec = 2. #0.5
#iti = 1.
#vols_per_trial = 15
#same_order = True
# =================================================================================


acquisition_dir = os.path.join(source, experiment, session, acquisition)
figdir = os.path.join(acquisition_dir, 'example_figures')


# Load reference info:
ref_json = 'reference_%s.json' % functional_dir 
with open(os.path.join(acquisition_dir, ref_json), 'r') as fr:
    ref = json.load(fr)

In [26]:
# =====================================================
# Set ROI method and Trace method:
# =====================================================
curr_roi_method = 'blobs_DoG' #ref['roi_id'] #'blobs_DoG'
#curr_trace_method = ref['trace_id'] #'blobs_DoG'
trace_dir = os.path.join(ref['trace_dir'], curr_roi_method)
#trace_dir = ref['trace_dir']
# =====================================================

# Create parsed-trials dir with default format:
parsed_traces_dir = os.path.join(trace_dir, 'Parsed')
if not os.path.exists(parsed_traces_dir):
    os.mkdir(parsed_traces_dir)


# Load SI meta data:
si_basepath = ref['raw_simeta_path'][0:-4]
simeta_json_path = '%s.json' % si_basepath
with open(simeta_json_path, 'r') as fs:
    simeta = json.load(fs)

file_names = sorted([k for k in simeta.keys() if 'File' in k], key=natural_keys)
nfiles = len(file_names)



In [27]:
# Get masks for each slice: 
roi_methods_dir = os.path.join(acquisition_dir, 'ROIs')
roiparams = loadmat(os.path.join(roi_methods_dir, curr_roi_method, 'roiparams.mat'))
maskpaths = roiparams['roiparams']['maskpaths']
if not isinstance(maskpaths, list):
    maskpaths = [maskpaths]

masks = dict(("Slice%02d" % int(slice_idx+1), dict()) for slice_idx in range(len(maskpaths)))
for slice_idx,maskpath in enumerate(sorted(maskpaths, key=natural_keys)):
    slice_name = "Slice%02d" % int(slice_idx+1)
    print "Loading masks: %s..." % slice_name 
    currmasks = loadmat(maskpath); currmasks = currmasks['masks']
    masks[slice_name]['nrois'] =  currmasks.shape[2]
    masks[slice_name]['masks'] = currmasks


Loading masks: Slice01...


In [28]:
# Get PARADIGM INFO:
path_to_functional = os.path.join(acquisition_dir, functional_dir)
paradigm_dir = 'paradigm_files'
path_to_paradigm_files = os.path.join(path_to_functional, paradigm_dir)

# Load stimulus dict:
stimdict_fn = 'stimdict.pkl'
with open(os.path.join(path_to_paradigm_files, stimdict_fn), 'r') as f:
     stimdict = pkl.load(f) #json.load(f)
print "STIMDICT: ", sorted(stimdict.keys(), key=natural_keys)


STIMDICT:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']


In [29]:
for stim in stimdict.keys():
    for filename in stimdict[stim].keys():
        for t in range(len(stimdict[stim][filename].frames)):
            if len(stimdict[stim][filename].frames[t])>270:
                print stim, filename, len(stimdict[stim][filename].frames[t])


In [30]:

# Split all traces by stimulus-ID:
# ----------------------------------------------------------------------------
stim_ntrials = dict()
for stim in stimdict.keys():
    stim_ntrials[stim] = 0
    for fi in stimdict[stim].keys():
        stim_ntrials[stim] += len(stimdict[stim][fi].trials)

# Load trace structs:
curr_tracestruct_fns = os.listdir(trace_dir)
trace_fns_by_slice = sorted([t for t in curr_tracestruct_fns if 'traces_Slice' in t], key=natural_keys)
#trace_fns_by_slice = sorted(ref['trace_structs'], key=natural_keys)

In [39]:
stimtraces_all_slices = dict()

for slice_idx,trace_fn in enumerate(sorted(trace_fns_by_slice, key=natural_keys)):

    currslice = "Slice%02d" % int(slice_idx+1)
    stimtraces = dict((stim, dict()) for stim in stimdict.keys())

    tracestruct = loadmat(os.path.join(trace_dir, trace_fn))
    
    # To look at all traces for ROI 3 for stimulus 1:
    # traces_by_stim['1']['Slice01'][:,roi,:]
    for stim in sorted(stimdict.keys(), key=natural_keys):
        #stimtraces[stim] = dict()
        repidx = 0
        curr_traces_allrois = []
        stim_on_frames = []
        for fi,currfile in enumerate(sorted(file_names, key=natural_keys)):
#            nframes = int(simeta[currfile]['SI']['hFastZ']['numVolumes'])
#            framerate = float(simeta[currfile]['SI']['hRoiManager']['scanFrameRate'])
            volumerate = float(simeta[currfile]['SI']['hRoiManager']['scanVolumeRate'])
#            frames_tsecs = np.arange(0, nframes)*(1/volumerate)

#            nframes_on = stim_on_sec * volumerate
#            nframes_off = vols_per_trial - nframes_on
#            frames_iti = round(iti * volumerate) 

            curr_ntrials = len(stimdict[stim][currfile].frames)
            currtraces = tracestruct['file'][fi].tracematDC
            for currtrial_idx in range(curr_ntrials):
                
                currtrial_frames = stimdict[stim][currfile].frames[currtrial_idx]
                #print len(currtrial_frames)
   
                # .T to make rows = rois, cols = frames 
                nframes = currtraces.shape[0]
                nrois = currtraces.shape[1] 
                print currtraces[currtrial_frames, :].shape
                curr_traces_allrois.append(currtraces[currtrial_frames, :])
                repidx += 1
                
                curr_frame_onset = stimdict[stim][currfile].stim_on_idx[currtrial_idx]
                stim_on_frames.append([curr_frame_onset, curr_frame_onset + stim_on_sec*volumerate])

        stimtraces[stim]['traces'] = np.asarray(curr_traces_allrois)
    	stimtraces[stim]['frames_stim_on'] = stim_on_frames 
        stimtraces[stim]['ntrials'] = stim_ntrials[stim]
        stimtraces[stim]['nrois'] = nrois


(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(268, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(268, 144)
(268, 144)
(268, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(269, 144)
(268, 144)
(269, 144)

In [40]:
#curr_traces_allrois.shape

y=np.dstack(curr_traces_allrois)
print y.shape
# (10, 10, 5)

# To get the shape to be Nx10x10, you could  use rollaxis:
y=np.rollaxis(y,-1)
print y.shape

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [33]:
for stim in stimtraces.keys():
    for t in range(len(stimtraces[stim]['traces'])):
        print stim, t, stimtraces[stim]['traces'][t].shape

11 0 (268, 144)
11 1 (269, 144)
11 2 (269, 144)
11 3 (268, 144)
11 4 (269, 144)
11 5 (269, 144)
11 6 (269, 144)
11 7 (269, 144)
11 8 (269, 144)
11 9 (269, 144)
11 10 (269, 144)
11 11 (269, 144)
11 12 (269, 144)
11 13 (269, 144)
11 14 (269, 144)
11 15 (269, 144)
11 16 (269, 144)
11 17 (269, 144)
11 18 (269, 144)
11 19 (269, 144)
10 0 (269, 144)
10 1 (269, 144)
10 2 (269, 144)
10 3 (269, 144)
10 4 (269, 144)
10 5 (269, 144)
10 6 (269, 144)
10 7 (269, 144)
10 8 (268, 144)
10 9 (269, 144)
10 10 (269, 144)
10 11 (269, 144)
10 12 (269, 144)
10 13 (269, 144)
10 14 (269, 144)
10 15 (269, 144)
10 16 (269, 144)
10 17 (268, 144)
10 18 (269, 144)
10 19 (269, 144)
12 0 (269, 144)
12 1 (269, 144)
12 2 (269, 144)
12 3 (269, 144)
12 4 (269, 144)
12 5 (269, 144)
12 6 (269, 144)
12 7 (269, 144)
12 8 (269, 144)
12 9 (269, 144)
12 10 (268, 144)
12 11 (269, 144)
12 12 (269, 144)
12 13 (269, 144)
12 14 (268, 144)
12 15 (268, 144)
12 16 (269, 144)
12 17 (269, 144)
12 18 (268, 144)
12 19 (269, 144)
1 0 (269, 